In [ ]:
import cobra
from cobra.io import save_json_model, read_sbml_model, write_sbml_model
import os, re
import pandas as pd

print(os.getcwd())

ext_dir = '/../../../data/external'
gap_uni_dir = '/../2.gapfill_w_universal/Salb-GEM-Uni-gapfill.xml'

# Import model
# model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM.xml")
# model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM-Updated.xml")
model = read_sbml_model(f"{os.getcwd()}/{gap_uni_dir}")
model_ori = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM-Updated.xml")

In [ ]:
model.reactions.NRRH

In [ ]:
for r in model.metabolites.ncam_c.reactions:
    print(r.build_reaction_string())

In [ ]:
model.metabolites.nac_c

In [ ]:
with model:
    print(model.slim_optimize())

# Get all gapfilled reactions

In [ ]:
diff_r = []
for r in model.reactions:
    if model_ori.reactions.has_id(r.id) == False:
        diff_r.append(r)

reaction_data = [{
    'Reaction ID': reaction.id,
    'Name': reaction.name,
    'Reaction String': reaction.reaction,
    'Reference': reaction.annotation
} for reaction in diff_r]

reaction_df = pd.DataFrame(reaction_data)
reaction_df.to_csv('all_gapfilled_reactions.csv')
print(reaction_df)

In [ ]:
# Find out what genes could be wrong by checking growth.

possibly_wrong = []
for r in diff_r:
    with model_ori:
        model_ori.add_reactions([r])
        model_ori.objective = 'growth'
        solution = model_ori.optimize().objective_value
    print(r, solution)
    if solution > 0.1:
        possibly_wrong.append(r)

possibly_wrong

In [ ]:
addition = list(set(diff_r) - set(possibly_wrong))

# Manual Check Gapfilled Reactions in all_gapfilled_reactions.xlsx

1. Possibly wrong reactions are removed from the list (DTDP26DGR, DHPS_2, SRK, SDPDS_2, THDPS_2)
2. Reaction names are updated
3. TODO: possibly matching a gene

In [ ]:
# Update reaction names
for r in addition:
    if r.name == "":
        print(r.id)


In [ ]:
# Reaction name is updated from kegg and ncbi
# https://www.ncbi.nlm.nih.gov/datasets/taxonomy/1883
model.reactions.NP1_2.name = 'nicotinate D-ribonucleoside:orthophosphate ribosyltransferase'
model.reactions.NP1_2.annotation = {'kegg': 'R02295'}

model.reactions.CPS_1.name = 'carbamoyl-phosphate synthase'
model.reactions.CPS_1.annotation = {'kegg':'R00149'}

model.reactions.NRRH.name = 'nicotinate D-ribonucleoside ribohydrolase'
model.reactions.NRRH.annotation = {'kegg': 'R10046'}

In [ ]:
with model_ori:
    model_ori.add_reactions(addition)
    media = model_ori.medium
    media['EX_glc__D_e'] = 0
    # media['EX_mnl_e'] = 0.8
    media['EX_glu__L_e'] = 0.96
    # media['EX_glyc_e'] = 1.6
    model_ori.medium = media
    model.objective = 'growth'
    solution = model_ori.optimize()

print(solution.objective_value)

In [ ]:
model_ori.add_reactions(addition)

In [ ]:
model_ori.reactions.EX_co2_e.bounds= (0, 1000)

cobra.io.write_sbml_model(model_ori, 'Salb-GEM-Man-gapfill.xml')